# VdM analysis - a minimal example

This is a brief introduction to the technicalities of the VdM method.  
It is not a entirely accurate description of the VdM Framework, but it is capable of producing same results given equal conditions.  

It is nicer to look at scans where there are only a few colliding bunches, as the fit plots are displayed in the notebook.

When in doubt, restart the kernel and run the whole program again.

You can find a pre-run notebook including output in minimal_example.html

*Prequisites:* Install the needed packages with `python3 -m pip install -U -r requirements.txt`

**Original author: Santeri Saariokari  
Thanks: Nimmitha Karunarathna**

In [ ]:
from pathlib import Path

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import tables
import matplotlib
import matplotlib.pyplot as plt
import mplhep as hep
from scipy import stats
from iminuit import Minuit
from iminuit.cost import LeastSquares

### Required "arguments"
* File name
* Luminometer name
* Fit type
* Flag to draw plots (T/F)
* Flag to calibrate beam current (T/F)

In [ ]:
filename = '/eos/cms/store/group/dpg_bril/comm_bril/vdmdata/2021/original/7525/7525_2110302352_2110310014.hd5'
fit = 'SG'
luminometer = 'pltlumizero'
plots = True  
calibrate_beam_current = True

### Define fit functions
* Single Gaussian
* Single Gaussian + constant

In [ ]:
def sg(x, peak, mean, cap_sigma):
    return peak*np.exp(-(x-mean)**2/(2*cap_sigma**2))

def sg_const(x, peak, mean, cap_sigma, constant):
    return sg(x, peak, mean, cap_sigma) + constant

Each function needs a mapping from string parameter, and also a set of initial conditions

In [ ]:
FIT_FUNCTIONS = {
    'SG':       {'handle': sg,       'initial_values': {'peak': 1e-4, 'mean': 0, 'cap_sigma': 0.3}},
    'SGConst': {'handle': sg_const, 'initial_values': {'peak': 1e-4, 'mean': 0, 'cap_sigma': 0.3, 'constant': 0}}
}

General settings and definitions

In [ ]:
%matplotlib inline
plt.style.use(hep.style.CMS)

outpath = f'output/{Path(filename).stem}' # Save output to this folder
Path(outpath).mkdir(parents=True, exist_ok=True) # Create output folder if not existing already

**H5 contains several tables under "/" (root group). Following tables will be used.**
* luminometer Ex: pltlumizero - *time, rate in each bunch*
* beam - *time, beam energy, beam intensity per bunch (from FBCT and DCCT)*
* vdmscan - *to get scan conditions: time, fill, run, lumiSec, nibble, ip, beam separation*

In [ ]:
f = tables.open_file(filename, 'r')
print(f)

## Create a dataframe with general info about the scan

**Using the tables "vdmscan" and "beam"**

In [ ]:
cols = f.root.vdmscan.colnames #Get colum names from the vdmscan table
print("\n".join(cols))

**Scan conditions are saved in table *vdmscan*  with timestamps. It is assumed that these are not going to change during a scan (at least the ones we care about)**

Get first row of table "vdmscan" to save scan conditions that are constant through the scan

In [ ]:
general_info = pd.DataFrame([list(f.root.vdmscan[0])], columns=cols)
general_info

**Get the scanning points as a list.**

IP is represented in a 8-bit binary number(Ex: 0b100000), which is read from the table as a decimal number(Ex: 32)

Convert dec to binary list all scanning IPs (scan in IP n iff bit n == 1)

Ex: 32 -> 0b100000 -> IP5

Note the potentially confusing situtation: 2 means ATLAS, as 0b000001 = 2^1 = 2  

In [ ]:
general_info['ip'].item()

In [ ]:
general_info['ip'] = general_info['ip'].apply(lambda ip: [i for i,b in enumerate(bin(ip)[::-1]) if b == '1'])

**Beam energy can be retrieved from the "beam" table**

In [ ]:
general_info['energy'] = f.root.beam[0]['egev']

In [ ]:
general_info = general_info[['fillnum', 'runnum', 'timestampsec', 'energy', 'ip', 'bstar5', 'xingHmurad']]
general_info

## Associate timestamps to pairs of scan planes and scan points

Get timestamps (timestampsec), beam seperation (sep) and scanning plane (nominal_sep_plane) when the *stat == ACQUIRING*

* CROSSING == X Plane
* SEPRARATION == Y Plane

In [ ]:
scan = pd.DataFrame()

scan['timestampsec'] = [r['timestampsec'] for r in f.root.vdmscan.where('stat == "ACQUIRING"')]
scan['sep'] = [r['sep'] for r in f.root.vdmscan.where('stat == "ACQUIRING"')]
scan['nominal_sep_plane'] = [r['nominal_sep_plane'].decode('utf-8') for r in f.root.vdmscan.where('stat == "ACQUIRING"')] # Decode is needed for values of type string
scan

**Group data by plane and then by separation. For each separation list the minumum and maximum timestamp. This basically list the beginning and end of each scan step**

In [ ]:
scan = scan.groupby(['nominal_sep_plane', 'sep']).agg(min_time=('timestampsec', np.min), max_time=('timestampsec', np.max)) # Get min and max for each plane - sep pair
scan

In [ ]:
scan.reset_index(inplace=True) # Cast <groupby> to normal dataframe
print('\nScanpoints and timestamps\n', scan)

## Collecting observable rates from the luminometer

**The column 'collidable' in f.root.beam contains colliding bunches as an array of length 3564 (0-indexed).**

In [ ]:
print(f.root.beam[0]['collidable'])

In [ ]:
collidable = np.nonzero(f.root.beam[0]['collidable'])[0]  # np.nonzero returns 2 arrays where the first contains indices
print("collidable", collidable)

**bxconfig1 and bxconfig2 contain the filled bunches in each beam. Logical OR will give all filled bunches**

In [ ]:
filled = np.nonzero(np.logical_or(f.root.beam[0]['bxconfig1'], f.root.beam[0]['bxconfig2']))[0] 
print("filled", filled)

### Go through each scan point and get rate and beam currect values for each colliding bunch

The column *bxraw* in f.root.pltlumizero contains rates of each bunch per integration time unit (NB4). It is needed to extract bxraw from the table based on the period of each seperation scan. This has to be done for each seperation for both planes

In [ ]:
print(f.root[luminometer][:]['bxraw'])
print(f.root[luminometer][:]['bxraw'].shape)

**For each plane & for each sep**
1. Calculate "period_of_scanpoint". This is a query with scan begin and end times. This will be used to query the "beam" table.
    
    Ex: (timestampsec > 1635638276) & (timestampsec <= 1635638305)

    **Do the following for the period_of_scanpoint**

2. Get rates for colliding bunches from luminometer table and take the mean
3. Calculate rate error using Standard Error of the Mean (SEM)
4. Get bunch intensity for beam 1 and 2 from beam table. Columns "bxintensity1" and "bxintensity2" and take the mean
5. Get beam intensities again using columns "intensity1" and "intensity2" and take the mean

---

**Why there is two separate sets of values for beam current:**  
We have two systems to measure beam current. FBCT gives intensity per bunch, and DCCT gives the sum over all bunches. And we want per bunch. But DCCT is more accurate.
So what is done, is bunch-by-bunch variation is taken from FBCT and normalized so that the sum equals DCCT value.
This is the FBCT/DCCT calibration mentioned in the framework

In [ ]:
data = pd.DataFrame()
for p, plane in enumerate(scan.nominal_sep_plane.unique()):
    for sep in scan.sep.unique():
        new = pd.DataFrame()
        new['bcid'] = collidable + 1 # From 0-indexed to 1-indexed
        
        # building the time query for the current separation 
        sep_start = scan.min_time[(scan.nominal_sep_plane == plane) & (scan.sep == sep)].item()
        sep_end = scan.max_time[(scan.nominal_sep_plane == plane) & (scan.sep == sep)].item()
        period_of_scanpoint = f'(timestampsec > {sep_start}) & (timestampsec <= {sep_end})'

        # Only get rate for colliding bunches
        r = np.array([r['bxraw'][collidable] for r in f.root[luminometer].where(period_of_scanpoint)]) 
        new['rate'] = r.mean(axis=0) # Mean over LS
        new['rate_err'] = stats.sem(r, axis=0)
        
        new['fbct1'] = np.array([b['bxintensity1'][collidable] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0)
        new['fbct2'] = np.array([b['bxintensity2'][collidable] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0)
        
        # DCCT is not per bunch, instead same value that contains the sum over BCIDs is repeated for all BCIDs
        new['dcct1'] = np.array([b['intensity1'] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0)
        new['dcct2'] = np.array([b['intensity2'] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0)
        
        # Additional quantities are needed for beam current calibration
        if calibrate_beam_current:
            fbct_filled1 = np.array([b['bxintensity1'][filled] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0).sum()
            fbct_filled2 = np.array([b['bxintensity2'][filled] for b in f.root['beam'].where(period_of_scanpoint)]).mean(axis=0).sum()
            new['fbct_to_dcct_beam1'] = fbct_filled1 / new['dcct1']
            new['fbct_to_dcct_beam2'] = fbct_filled2 / new['dcct2']

        new.insert(0, 'sep', sep) # Inserting constant as a column value will fill the column with the value
        new.insert(0, 'plane', plane)
        data = pd.concat([data, new])

In [ ]:
data.reset_index(drop=True, inplace=True)
data.head()

#### Normalize the rate by the product of beam currents

In [ ]:
beam = data['fbct1'] * data['fbct2'] / 1e22  # dividing by 1e22 produces the desired units for sigvis in the end
data['rate_normalised'] = data.rate / beam
data['rate_normalised_err'] = data.rate_err / beam

Calibrate beam current if specified

In [ ]:
if calibrate_beam_current:
    calib = data.groupby('plane')[['fbct_to_dcct_beam1', 'fbct_to_dcct_beam2']].mean()
    print('\nFBCT to DCCT calibration coefficients\n', calib)
    
    calib = calib.prod(axis=1) # Mean over LS, prod over beams
    print("\nProduct over beams\n",calib)
    
    for p in calib.index:
        data.loc[data.plane == p, 'rate_normalised'] *= calib[calib.index == p].item()
        data.loc[data.plane == p, 'rate_normalised_err'] *= calib[calib.index == p].item()

Add sensible error in case of 0 rate: max of error. The uncertainty would otherwise be 0, which is not true

In [ ]:
data['rate_normalised_err'].replace(0, data['rate_normalised_err'].max(), inplace=True)
data.to_csv(f'{outpath}/{luminometer}_data.csv', index=False)

In [ ]:
data.head()

Make a fit for each BCID in both planes

In [ ]:
fit_results = pd.DataFrame()
for p, plane in tqdm(enumerate(data.plane.unique())): # For each plane
    for bcid in tqdm(collidable+1, leave=False): # For each BCID
        data_x = scan[scan.nominal_sep_plane == plane]['sep']
        data_y = data[(data.plane == plane) & (data.bcid == bcid)]['rate_normalised']
        data_y_err = data[(data.plane == plane) & (data.bcid == bcid)]['rate_normalised_err']
        
        # Initialise minimiser with data and fit function of choice
        least_squares = LeastSquares(data_x, data_y, data_y_err, FIT_FUNCTIONS[fit]['handle'])
        m = Minuit(least_squares, **FIT_FUNCTIONS[fit]['initial_values']) # Initial values defined in "FIT_FUNCTIONS"
        
        m.migrad()  # Finds minimum of least_squares function
        m.hesse()  # Accurately computes uncertainties
        
        print(m)
        
        new = pd.DataFrame([m.values], columns=m.parameters) # Store values to dataframe
        
        # Add suffix "_err" to errors
        new = pd.concat([new, pd.DataFrame([m.errors], columns=m.parameters).add_suffix('_err')], axis=1)
        
        # Save fit status
        new['valid'] =  m.valid
        new['accurate'] = m.accurate
        
        new.insert(0, 'bcid', bcid)
        new.insert(0, 'plane', plane)
        fit_results = pd.concat([fit_results, new], ignore_index=True)
        
        # From here on just plotting
        if plots:
            # Initialise upper part: fit and data points
            fig = plt.figure()
            ax1 = fig.add_axes((.12,.3,.83,.65))

            hep.cms.label(llabel="Preliminary",
                rlabel=fr"Fill {general_info.fillnum[0]}, $\sqrt{{s}}={general_info['energy'][0]*2/1000:.1f}$ TeV", loc=1)

            ax1.set_ylabel('$R/(N_1 N_2)$ [arb.]')
            ax1.set_xticklabels([])
            ax1.ticklabel_format(axis='y', style='sci', scilimits=(0,0), useMathText=True, useOffset=False)
            ax1.minorticks_off()

            # Initialise lower part: residuals
            ax2 = fig.add_axes((.12,.1,.83,.2))
            ax2.ticklabel_format(axis='y', style='plain', useOffset=False)
            ax2.set_ylabel('Residual [$\sigma$]',fontsize=20)
            ax2.set_xlabel('$\Delta$ [mm]')
            ax2.minorticks_off()
            
            # Plot the data points
            figure_items = []
            figure_items.append(ax1.errorbar(data_x, data_y, data_y_err, fmt='ko'))
            x_dense = np.linspace(np.min(data_x), np.max(data_x))
            
            # Plot the fit result
            figure_items.append(ax1.plot(x_dense, FIT_FUNCTIONS[fit]['handle'](x_dense, *m.values), 'k'))

            fit_info = [f'{plane}, BCID {bcid}', f'$\\chi^2$ / $n_\\mathrm{{dof}}$ = {m.fval:.1f} / {len(data_x) - m.nfit}']
            for param, v, e in zip(m.parameters, m.values, m.errors):
                fit_info.append(f'{param} = ${v:.3e} \\pm {e:.3e}$')

            fit_info = [info.replace('cap_sigma', '$\Sigma$') for info in fit_info]

            figure_items.append(ax1.text(0.95, 0.95, '\n'.join(fit_info), transform=ax1.transAxes, fontsize=14, fontweight='bold',
                verticalalignment='top', horizontalalignment='right'))

            residuals = (data_y.to_numpy() - FIT_FUNCTIONS[fit]['handle'](data_x, *m.values).to_numpy()) / data_y_err.to_numpy()
            figure_items.append(ax2.scatter(data_x, residuals, c='k'))
            
            # Plot long "zero-line" without changing xlim
            lim = list(plt.xlim())
            figure_items.append(ax2.plot(lim, [0, 0], 'k:'))
            plt.xlim(lim)
            
            plt.show()

            # Only delete lines and fit results, leave general things
            for item in figure_items:
                if isinstance(item, list):
                    item[0].remove()
                else:
                    item.remove()

In [ ]:
fit_results.cap_sigma *= 1e3 # to µm
fit_results.cap_sigma_err *= 1e3 # to µm
fit_results.to_csv(f'{outpath}/{luminometer}_fit_results.csv', index=False)
fit_results.head()

In [ ]:
fr = fit_results.pivot(index='bcid', columns=['plane'], values=['cap_sigma', 'peak', 'cap_sigma_err', 'peak_err'])
fr

 $\sigma_{vis} = 2\pi\Sigma_x\Sigma_y\frac{peak_1+peak_2}{2}$

In [ ]:
sigvis = np.pi * fr.cap_sigma.prod(axis=1) * fr.peak.sum(axis=1)

Propagation of uncertainty: $\Delta(\sigma_{vis}) = \sigma_{vis}\sqrt{\frac{\Delta(\Sigma_x)^2}{\Sigma_x^2} + \frac{\Delta(\Sigma_y)^2}{\Sigma_y^2} + \frac{\Delta(peak_x)^2 + \Delta(peak_y)^2}{(peak_x+peak_y)^2}}$

In [ ]:
sigvis_err = (fr.cap_sigma_err**2 / fr.cap_sigma**2).sum(axis=1) + (fr.peak_err**2).sum(axis=1) / (fr.peak).sum(axis=1)**2
sigvis_err = np.sqrt(sigvis_err) * sigvis
lumi = pd.concat([sigvis, sigvis_err], axis=1)
lumi.columns = ['sigvis', 'sigvis_err']
lumi.to_csv(f'{outpath}/{luminometer}_lumi.csv')
lumi